**Import Libraries**

In [1]:
import tensorflow as tf
import numpy as np
import warnings
import os
import random
random.seed(42)
np.random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
tf.random.set_seed(42)
from nlp_model_text_preprocessing import index_the_words, text_to_sequence, pad_sequences, one_hot_encoding, index_the_char, text_to_sequence_char, char_sequence_to_text, word_sequence_to_text
from arabic_text_normalization import text_normalization
from deep_learning import nlp_model_word, nlp_model_char, model_compile_word, model_compile_char, model_fit, plot_word_model_change, plot_char_model_change
from model_testing import model_testing_char, model_testing_word
from transformers_models import load_dataset, data_collator, train_arguments, training_, save_model_tokenizer, transformer_testing,transformes_model
from read_data import read_file
from generate_train_label import generate_train_label_word, generate_train_label_char
from model_check_point import check_point
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

**Ignore Warnings**

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='ignore')

**Read Data**

In [3]:
Corpus = read_file(file_path='الخيميائي.txt', text_normalization=text_normalization)
len(Corpus)

1242

In [4]:
with open('Cleaned_Corpus.txt', 'w') as f:
    for line in Corpus:
        f.write(line + '\n') 

**Generate Early Stop Depending On Value Of Loss**

In [5]:
early_stop_ = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=5,
    mode = 'min',
    restore_best_weights=True,
)

**1-Tokens Based On Word**

In [ ]:
# Generate Word To Index, Index To Word, And Find Count Of All Words
all_words, words_index, index_to_words = index_the_words(Corpus)
all_words

In [ ]:
max_length_word, train, labels = generate_train_label_word(Corpus, text_to_sequence, words_index, pad_sequences)
max_length_word

In [ ]:
# Convert Label Data 
label = one_hot_encoding(labels, all_words)
label

*Deep Learning Models*

In [ ]:
# 1-LSTM
arabic_lstm_check_point_1 = check_point('Arabic_Lstm_1')
lstm_model = tf.keras.layers.LSTM(units=128, return_sequences=False)
LSTM_1 = nlp_model_word(input_dim = all_words, output_dim = 100, input_length = max_length_word, unit = all_words, model = lstm_model)
model_compile_word(model =LSTM_1, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
history = model_fit(model=LSTM_1, Data=train, Label=label, epochs=150, early_stop=early_stop_, checkpoint=arabic_lstm_check_point_1 ,batch_size=32)
plot_word_model_change(history=history)

In [ ]:
# 2-Bidirectional LSTM
arabic_bidirectional_lstm_check_point_1 = check_point('Arabic_Bidirectional_1')
bidirectional_lstm_model = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units= 128, return_sequences=False))
Bidirectional_LSTM_1 = nlp_model_word(input_dim = all_words, output_dim = 100, input_length = max_length_word, unit = all_words, model = bidirectional_lstm_model)
model_compile_word(model =Bidirectional_LSTM_1, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
history = model_fit(model=Bidirectional_LSTM_1, Data=train, Label=label, epochs=150, early_stop=early_stop_, checkpoint=arabic_bidirectional_lstm_check_point_1, batch_size=64)
plot_word_model_change(history=history)

In [ ]:
# 3-GRU
arabic_gru_check_point_1 = check_point('Arabic_GRU1')
gru_model = tf.keras.layers.GRU(units= 128, return_sequences=False)
GRU_1 = nlp_model_word(input_dim = all_words, output_dim = 100, input_length = max_length_word, unit = all_words, model = gru_model)
model_compile_word(model =GRU_1, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
history = model_fit(model=GRU_1, Data=train, Label=label, epochs=150, early_stop=early_stop_, checkpoint=arabic_gru_check_point_1 , batch_size=32)
plot_word_model_change(history=history)

*Testing The Model*

In [ ]:
# 1-LSTM
model_testing_word(text = 'تناول الخيميائي ', number_of_words = 50, text_normalization = text_normalization, text_to_sequence = text_to_sequence, words_index = words_index, pad_sequences = pad_sequences, checkpoint_filepath='Arabic_Lstm_1_model_checkpoint.h5', max_length = max_length_word, index_to_words = index_to_words, word_sequence_to_text = word_sequence_to_text, all_words=all_words)

In [ ]:
# 2-Bidirectional LSTM
model_testing_word(text = 'رأى تاجر الزجاجيّات شروق ', number_of_words = 50, text_normalization = text_normalization, text_to_sequence = text_to_sequence, words_index = words_index, pad_sequences = pad_sequences, checkpoint_filepath='Arabic_Bidirectional_1_model_checkpoint.h5', max_length = max_length_word, index_to_words = index_to_words, word_sequence_to_text = word_sequence_to_text, all_words=all_words)

In [ ]:
# 3-GRU
model_testing_word(text = 'إن مافهمه في تلك اللحظة ', number_of_words = 50, text_normalization = text_normalization, text_to_sequence = text_to_sequence, words_index = words_index, pad_sequences = pad_sequences, checkpoint_filepath='Arabic_GRU1_model_checkpoint.h5', max_length = max_length_word, index_to_words = index_to_words, word_sequence_to_text = word_sequence_to_text, all_words=all_words)

**2-Token Based On Character**

In [ ]:
Corpus = Corpus[0:5]
Corpus

In [6]:
# Generate Char To Index, Index To Char, And Find Count Of All Char
all_chars, chars_index, index_chars = index_the_char(Corpus)
all_chars

45

In [7]:
chars_index

{' ': 0,
 'ء': 1,
 'آ': 2,
 'أ': 3,
 'ؤ': 4,
 'إ': 5,
 'ئ': 6,
 'ا': 7,
 'ب': 8,
 'ة': 9,
 'ت': 10,
 'ث': 11,
 'ج': 12,
 'ح': 13,
 'خ': 14,
 'د': 15,
 'ذ': 16,
 'ر': 17,
 'ز': 18,
 'س': 19,
 'ش': 20,
 'ص': 21,
 'ض': 22,
 'ط': 23,
 'ظ': 24,
 'ع': 25,
 'غ': 26,
 'ف': 27,
 'ق': 28,
 'ك': 29,
 'ل': 30,
 'م': 31,
 'ن': 32,
 'ه': 33,
 'و': 34,
 'ى': 35,
 'ي': 36,
 'ً': 37,
 'ٌ': 38,
 'ٍ': 39,
 'َ': 40,
 'ُ': 41,
 'ِ': 42,
 'ّ': 43,
 'ْ': 44,
 'UNK': 45}

In [8]:
# Find The Max Length
max_length_char = max([len(s) for s in Corpus])
max_length_char

866

In [9]:
# Generate Train And Label Data
Train, Label = generate_train_label_char(Corpus, max_length_char)

In [10]:
# Convert Words Into Number
sequence_text_char_Train = text_to_sequence_char(chars_index, Train)
sequence_text_char_Label = text_to_sequence_char(chars_index, Label)

In [ ]:
# Padding The The Input Sequence To Make All Sequence In Same Length
Train = pad_sequences(input_sequence=sequence_text_char_Train, max_length=max_length_char, padding='post')
# # Convert The Label Data
Label = one_hot_encoding(sequence_text_char_Label, all_chars)

**Deep Learning Model**

In [ ]:
# 1-LSTM
arabic_bidirectional_lstm_check_point_2 = check_point('Arabic_Bidirectional_2')
lstm_model = tf.keras.layers.LSTM(units= 1024, return_sequences=False)
LSTM_2 = nlp_model_char(input_dim = all_chars, output_dim = 100, unit = all_chars, model = lstm_model, input_length=max_length_char)
model_compile_char(model =LSTM_2, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy())
history = model_fit(model=LSTM_2, Data=Train, Label=Label, epochs=150, early_stop=early_stop_,  batch_size=32, checkpoint=arabic_bidirectional_lstm_check_point_2)
plot_char_model_change(history=history)

In [11]:
seq_length = max_length_char
examples_per_epoch = len(Corpus)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(sequence_text_char_Train)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
dataset = char_dataset.map(split_input_target)
BATCH_SIZE = 128
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 865), dtype=tf.int32, name=None), TensorSpec(shape=(128, 865), dtype=tf.int32, name=None))>

In [ ]:
examples_per_epoch

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),                         
    tf.keras.layers.LSTM(rnn_units, 
                          return_sequences=True,
                          stateful=True, recurrent_initializer='glorot_uniform'),                     
    
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM( batch_input_shape=[batch_size, None])),                                     
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [13]:
model = build_model(
  vocab_size = all_chars,
  embedding_dim=100,
  rnn_units=128,
  batch_size=BATCH_SIZE)

In [14]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [15]:
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [16]:
model.compile(optimizer='adam', loss=loss)

In [17]:
history = model.fit(dataset.repeat, epochs=300, callbacks=[checkpoint_callback], steps_per_epoch=7)

Epoch 1/300
7/7 [==============================] - 4s 315ms/step - loss: 3.7692
Epoch 2/300
7/7 [==============================] - 2s 312ms/step - loss: 3.3304
Epoch 3/300
7/7 [==============================] - 2s 326ms/step - loss: 3.1083
Epoch 4/300
7/7 [==============================] - 2s 305ms/step - loss: 3.0672
Epoch 5/300
7/7 [==============================] - 2s 299ms/step - loss: 3.0526
Epoch 6/300
7/7 [==============================] - 2s 299ms/step - loss: 3.0408
Epoch 7/300
7/7 [==============================] - 2s 358ms/step - loss: 3.0296
Epoch 8/300
7/7 [==============================] - 3s 461ms/step - loss: 3.0098
Epoch 9/300
7/7 [==============================] - 3s 413ms/step - loss: 2.9858
Epoch 10/300
7/7 [==============================] - 3s 379ms/step - loss: 2.9545
Epoch 11/300
7/7 [==============================] - 2s 317ms/step - loss: 2.9142
Epoch 12/300
7/7 [==============================] - 2s 330ms/step - loss: 2.8693
Epoch 13/300
7/7 [===================

In [ ]:
# 2-Bidirectional_LSTM
arabic_bidirectional_lstm_check_point_2 = check_point('Arabic_Bidirectional_2')
bidirectional_lstm_model = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units= 128, return_sequences=False))
Bidirectional_LSTM_2 = nlp_model_char(input_dim = all_chars, output_dim = 100, unit = all_chars, model = bidirectional_lstm_model, input_length=max_length_char)
model_compile_char(model =Bidirectional_LSTM_2, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy())
history = model_fit(model=Bidirectional_LSTM_2, Data=Train, Label=Label, epochs=150, early_stop=early_stop_, batch_size=32, checkpoint=arabic_bidirectional_lstm_check_point_2)
plot_char_model_change(history=history)

In [ ]:
# 3-GRU
gru_model = tf.keras.layers.GRU(units= 128, return_sequences=False)
GRU_2 = nlp_model_char(input_dim = all_chars, output_dim = 100, unit = all_chars, model = gru_model, input_length=max_length_char)
model_compile_char(model =GRU_2, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy())
history = model_fit(model=GRU_2, Data=Train, Label=Label, epochs=150, early_stop=early_stop, batch_size=32)
plot_char_model_change(history=history)

In [ ]:
import itertools
def model_testing_char(text, number_of_chars, char_index, pad_sequences, max_length, index_char, text_to_sequence_char, all_chars, char_sequence_to_text, checkpoint_filepath):
    model = tf.keras.models.load_model(checkpoint_filepath)
    encoded_text = text_to_sequence_char(char_index, text)
    encoded_text = list(list(itertools.chain.from_iterable(encoded_text)))
    padded_text = pad_sequences(padding='post', input_sequence=[encoded_text], max_length=max_length)
    for _ in range(number_of_chars):
        predicted_probs = model.predict(padded_text, verbose=0)
        index = np.argmax(predicted_probs)
        text += char_sequence_to_text(index_char, [index], all_chars)
        padded_text = np.append(padded_text, index)
        padded_text = pad_sequences(padding='post', input_sequence=[padded_text], max_length=max_length)
    return text

**Model Testing**

In [ ]:
# 1-LSTM
model_testing_char(text = 'تناول الخيميائي', number_of_chars = 200, char_index = chars_index, pad_sequences = pad_sequences, checkpoint_filepath= 'Arabic_Bidirectional_2_model_checkpoint.h5', max_length = max_length_char, index_char = index_chars, text_to_sequence_char = text_to_sequence_char, all_chars=all_chars, char_sequence_to_text=char_sequence_to_text)

In [ ]:
# 2-Bidirectional LSTM
model_testing_char(text = 'أضاءت أشعة القمر ', number_of_chars = 200, char_index = chars_index, pad_sequences = pad_sequences, model = Bidirectional_LSTM_2, max_length = max_length_char, index_char = index_chars, text_to_sequence_char = text_to_sequence_char, all_chars=all_chars, char_sequence_to_text=char_sequence_to_text)

In [ ]:
# 3-GRU
model_testing_char(text = 'أضاءت أشعة القمر ', number_of_chars = 200, char_index = chars_index, pad_sequences = pad_sequences, model = GRU_2, max_length = max_length_char, index_char = index_chars, text_to_sequence_char = text_to_sequence_char, all_chars=all_chars, char_sequence_to_text=char_sequence_to_text)

**3-Transformers**

In [ ]:
# Transformers Model Name
import torch
model_name_ = 'gpt2'
# Model And Tokenizer For Transformers
tokenizer, model = transformes_model(model_name_)
# Data Used For Train Transformers
train_dataset = load_dataset('Cleaned_Corpus.txt', tokenizer)
# Convert Data Into Batches
collator = data_collator(tokenizer)
# Set Arguments For Train The Transformers
train_args = train_arguments(epochs = 1)
# Train The Transformers
training_(model=model, training_args=train_args, collator = collator, data = train_dataset)
# Save The Model
save_model_tokenizer(model = model, tokenizer = tokenizer)
# Testing The Transformers Model
transformer_testing(input_text='إنني خيميائي ', tokenizer=tokenizer, model=model, text_normalization = text_normalization)
 